# Demonstration of Expand-T
In this Jupyter Notebook, we show all functionalities of Expand-T. 
Given a Source Table and a data lake, Expand-T uses [Gen-T](https://github.com/northeastern-datalab/gen-t) to first reclaim the source table. To do so, Expand-T first finds the set of originating tables from the data lake whose data values contribute to source values. Then, Expand-T performs integration in a targeted manner and reproduces the Source Table. Afterwards, the user can choose to expand their reclaimed table and see what additional attributes and tuples they can add to their source table, and what non-null values they can use to replace nulls. Lastly, users can sort and refine tuples in the expanded table to then input into their downstream task.

## Parameters and Imports
Users can import a source table and a data lake (benchmark). For now, we use a table from the [T2D Gold benchmark](https://webdatacommons.org/webtables/goldstandard.html).
User can also set a parameter for timeout at integration time. For now, we set the integration timeout to 1hr.

In [1]:
from IPython.display import display
import sys
import os
os.chdir('/home/gfan/gen-t-demo/')

In [2]:
# Parameters
benchmark = "t2d_gold"
source_table_name = "table_471_1.csv"
BENCHMARKPATH = '/home/gfan/Datasets/%s/' % (benchmark)  
integration_timeout = 3600

## Finding Candidate Tables
Expand-T uses Gen-T to find a set of candidate tables from the data lake. Gen-T uses an efficient off-the-shelf table discovery method to do so, like [Starmie](https://www.vldb.org/pvldb/vol16/p1726-fan.pdf). Then, Gen-T filters out the retrieved set of relevant tables to those that have high set overlap with the source table, and return these tables as **Candidate Tables**.

In [3]:
sys.path.append("discovery/")
from discovery_utils import get_lake, get_starmie_candidates
from discover_candidates import CandidateTables

runStarmie = 0
lake_dfs, all_lake_table_cols = get_lake(benchmark)
source_candidates = []
if runStarmie: 
    starmie_candidates = get_starmie_candidates(benchmark)
    source_candidates = starmie_candidates[source_table_name.replace('.csv', '')]

# Call CandidateTables to find candidates
candidate_table_finder = CandidateTables(benchmark,lake_dfs, all_lake_table_cols, source_candidates)
candidateTablesFound, _ = candidate_table_finder.find_candidates(source_table_name)
print(f"{len(candidateTablesFound)} Candidate Tables found for Source Table {source_table_name}")
print(list(candidateTablesFound.keys()))

12 Candidate Tables found for Source Table table_471_1.csv
['table_536.csv', 'table_50.csv', 'table_187.csv', 'table_267.csv', 'table_594.csv', 'table_25.csv', 'table_244.csv', 'table_95.csv', 'table_640.csv', 'table_292.csv', 'table_674.csv', 'table_760.csv']


## Finding Originating Tables
Gen-T prunes the set of candidate tables to only include tables that when integrated, reproduce the source table. We call this pruned set, **Originating Tables**.


In [4]:

from prune_candidates import OriginatingTables
# Call OriginatingTables to prune candidates to a set of originating tables
originating_tables_finder = OriginatingTables(benchmark, candidateTablesFound)
originating_tables, matrix_traversal_runtimes = originating_tables_finder.find_originating_tables(source_table_name)
source_df = originating_tables_finder.source_table
print(f"From {len(candidateTablesFound)} candidate tables, found {len(originating_tables)} originating tables")
print(originating_tables)
origin_tables_matched_cols = {}
if originating_tables: origin_tables_matched_cols = {t: candidateTablesFound[t] for t in originating_tables}


From 12 candidate tables, found 6 originating tables
['table_50.csv', 'table_267.csv', 'table_95.csv', 'table_674.csv', 'table_25.csv', 'table_244.csv']


## Reclaiming the Source Table
Gen-T now integrates the set of originating tables with the goal of reproducing or reclaiming the source table. We show the output of Table Reclamation, a **reclaimed Source Table**. This reclaimed table has bolded values highlighted in yellow for reproduced source values. For non-null values that are suggested to replace missing values in the source table, we underline them and highlight them in blue.

In [5]:
os.chdir('integration/')

In [6]:
from targeted_integration import TableIntegration
import integration_utils as utils 
sys.path.append('../discovery/')

table_integrator = TableIntegration(benchmark, origin_tables_matched_cols, integration_timeout)
timed_out, noCandidates, numOutputVals = table_integrator.integrate_tables(source_table_name)
integration_result = table_integrator.reproducedSourceTable
integration_result = table_integrator.order_rows_cols(integration_result)
integration_result_highlighted = table_integrator.highlight_overlap_values(integration_result)
display(integration_result_highlighted)

-----x---------x--------x---
Source Table:


,Title,Fans' Rank,Year,Director(s),Overall Rank
0,Pulp Fiction,2,1994,Quentin Tarantino,15
1,Citizen Kane,3,1941,Orson Welles,2
2,Star Wars,5,1977,NaN,8


-----x---------x--------x---
Reclaimed Source Table:


,Title,Fans' Rank,Year,Director(s),Overall Rank
0,Pulp Fiction,2,1994,Quentin Tarantino,15
1,Citizen Kane,3,1941,Orson Welles,2
2,Star Wars,5,1977,George Lucas,8


## Expanding the Source Table
Expand-T can also expand the reclaimed source table. Expand-T can suggest new data as follows:
* expand tables in a 'horizontal' direction: Expand-T adds more columns
* expand tables in a 'vertical' direction: Expand-T adds more rows
* expand tables in 'all' directions: Expand-T adds more columns and more rows

When expanding the table, the user can also specify the number $N$ of new columns and/or rows they would like to see. 

In [7]:
expanded_integ_result, new_cols, new_tuples = table_integrator.expand_tables('all', N=3)
expanded_integration_result_highlighted = table_integrator.highlight_overlap_values(expanded_integ_result)
display(expanded_integration_result_highlighted)


Integrated Table has 2 new columns: ['Religion', 'Actor']
Integrated Table has 414 new tuples


,Title,Fans' Rank,Year,Director(s),Overall Rank,Religion,Actor
0,Pulp Fiction,2,1994,Quentin Tarantino,15,Atheist,John Travolta
1,Citizen Kane,3,1941,Orson Welles,2,nan,Orson Welles
2,Star Wars,5,1977,George Lucas,8,nan,nan
3,Kill Bill: Vol. 2,224,2004,Quentin Tarantino,701,nan,nan
4,Night of the Living Dead,229,1968,George A. Romero,139,Christian,nan
5,The Piano,307,1993,Jane Campion,247,Christian,nan


## Exploring the Expanded Table
User can also sort and refine tuples in the expanded table. 

To sort tuples, they can call ``sort_tuples()`` and specify the column they would like to sort by (``sortCol``).

To refine or filter tuples, users can create a dictionary of column name and column values that they would like to filter by. Then, they can call ``filter_tuples()`` and pass in the dictionary.

In [8]:
sorted_df = table_integrator.sort_tuples(expanded_integ_result, sortCol='Overall Rank')
sorted_df = table_integrator.highlight_overlap_values(sorted_df)
display(sorted_df)

,Title,Fans' Rank,Year,Director(s),Overall Rank,Religion,Actor
0,Citizen Kane,3,1941,Orson Welles,2,nan,Orson Welles
1,Star Wars,5,1977,George Lucas,8,nan,nan
2,Pulp Fiction,2,1994,Quentin Tarantino,15,Atheist,John Travolta
3,Night of the Living Dead,229,1968,George A. Romero,139,Christian,nan
4,The Piano,307,1993,Jane Campion,247,Christian,nan
5,Kill Bill: Vol. 2,224,2004,Quentin Tarantino,701,nan,nan


In [9]:
filter_dict = {'Director(s)': ['Quentin Tarantino'], 
               'Overall Rank': [i for i in range(1000)]}
filtered_df = table_integrator.filter_tuples(expanded_integ_result, filter_dict)
filtered_df = table_integrator.highlight_overlap_values(filtered_df)
display(filtered_df)

,Title,Fans' Rank,Year,Director(s),Overall Rank,Religion,Actor
0,Pulp Fiction,2,1994,Quentin Tarantino,15,Atheist,John Travolta
1,Kill Bill: Vol. 2,224,2004,Quentin Tarantino,701,nan,nan


# Expand-T vs. Outer Join
Another method to reclaiming and expanding a source table is performing Outer Join on all candidate tables retrieved from the data lake. Instead of applying Gen-T to reclaim the table and expanding the table, we can directly integrate all candidate tables. However, as seen below, this integration method leads to many noisy tuples with nulls. Specifically, using outer join produces almost 2X as many tuples as Expand-T. 

In [10]:
joined_df = utils.outerjoin(list(originating_tables_finder.candidate_table_dfs.values()))
joined_df = table_integrator.order_rows_cols(joined_df)
joined_df = table_integrator.highlight_overlap_values(joined_df)
display(joined_df)

,Title,Fans' Rank,Year,Director(s),Overall Rank,Religion,Actor
0,Pulp Fiction,nan,1994,nan,nan,nan,nan
1,Pulp Fiction,2,1994,Quentin Tarantino,15,Atheist,nan
2,Pulp Fiction,nan,1994,Quentin Tarantino,15,nan,John Travolta
3,Citizen Kane,3,1941,Orson Welles,2,nan,nan
4,Citizen Kane,nan,1941,nan,nan,nan,nan
5,Citizen Kane,nan,1941,Orson Welles,2,nan,Orson Welles
6,Star Wars,nan,1977,George Lucas,8,nan,nan
7,Star Wars,5,1977,George Lucas,8,nan,nan
8,The Piano,307,1993,Jane Campion,247,Christian,nan
9,Night of the Living Dead,229,1968,George A. Romero,139,Christian,nan
